## 2021: Week 16 - The Super League

If you are a fan of football or have been following any of the latest news then you would have probably heard about the proposed new 'Super League' that was planned with a selection of European clubs. This would have involved a group of 12 teams playing in a competition each year without having to qualify or be relegated. The lack of  fair competition between other clubs has caused an uproar among fans, players, media outlets, football associations, and even the UK government! 

The 'big 6' English teams to propose the Super League were Arsenal, Chelsea, Liverpool, Manchester United, Manchester City, and Tottenham Hotspur.

One of the ideas to try and discourage the clubs from proceeding with the new league, was to threaten the English teams with being expelled from the English Premier League. The challenge this week is to try and understand how the current league table would change if these clubs were to be 'kicked out'.

### Input
The input this week is a list of all of the fixtures from the 2020/21 season (up until 19/04/2021). 

### Requirement
- Input the data
- Calculate the Total Points for each team. The points are as follows: 
    - Win - 3 Points
    - Draw - 1 Point
    - Lose - 0 Points
- Calculate the goal difference for each team. Goal difference is the difference between goals scored and goals conceded. 
- Calculate the current rank/position of each team. This is based on Total Points (high to low) and in a case of a tie then Goal Difference (high to low).
- The current league table is our first output.

- Assuming that the 'Big 6' didn't play any games this season, recalculate the league table.
- After removing the 6 clubs, how has the position changed for the remaining clubs?
- The updated league table is the second output.
- Bonus - Think about features in Tableau Prep to make this repeatable process easier!

### Output
1. Current League Table
    - 5 Fields, 20 Rows (21 including header)
    - Position
    - Team
    - Total Games Played
    - Total Points
    - Goal Difference
    
    
2. Updated League Table
    - 5 Fields, 14 Rows (15 including header)
    - Position
    - Team
    - Total Games Played
    - Total Points
    - Goal Difference

In [1]:
import pandas as pd
import numpy as np
import matplotlib.pyplot as plt
import seaborn as sns

In [ ]:
### Input the data

In [5]:
df = pd.read_csv("./data/PL Fixtures.csv", parse_dates=["Date"])
df.head()

,Round Number,Date,Location,Home Team,Away Team,Result
0,1,2020-12-09 12:30:00,Craven Cottage,Fulham,Arsenal,0 - 3
1,1,2020-12-09 15:00:00,Selhurst Park,Crystal Palace,Southampton,1 - 0
2,1,2020-12-09 17:30:00,Anfield,Liverpool,Leeds,4 - 3
3,1,2020-12-09 20:00:00,London Stadium,West Ham,Newcastle,0 - 2
4,1,2020-09-13 14:00:00,The Hawthorns,West Brom,Leicester,0 - 3


In [6]:
df.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 380 entries, 0 to 379
Data columns (total 6 columns):
 #   Column        Non-Null Count  Dtype         
---  ------        --------------  -----         
 0   Round Number  380 non-null    int64         
 1   Date          380 non-null    datetime64[ns]
 2   Location      380 non-null    object        
 3   Home Team     380 non-null    object        
 4   Away Team     380 non-null    object        
 5   Result        316 non-null    object        
dtypes: datetime64[ns](1), int64(1), object(4)
memory usage: 17.9+ KB


In [ ]:
### Calculate the Total Points for each team. The points are as follows

In [37]:
result_null = df.loc[df["Result"].isna()].index
df = df.drop(result_null, axis=0)
df["Home Score"] = df["Result"].map(lambda x: x.split("-")[0])
df["Away Score"] = df["Result"].map(lambda x: x.split("-")[-1])
df

,Round Number,Date,Location,Home Team,Away Team,Result,Home Score,Away Score
0,1,2020-12-09 12:30:00,Craven Cottage,Fulham,Arsenal,0 - 3,0,3
1,1,2020-12-09 15:00:00,Selhurst Park,Crystal Palace,Southampton,1 - 0,1,0
2,1,2020-12-09 17:30:00,Anfield,Liverpool,Leeds,4 - 3,4,3
3,1,2020-12-09 20:00:00,London Stadium,West Ham,Newcastle,0 - 2,0,2
4,1,2020-09-13 14:00:00,The Hawthorns,West Brom,Leicester,0 - 3,0,3
...,...,...,...,...,...,...,...,...
312,32,2021-04-17 12:30:00,St. James' Park,Newcastle,West Ham,3 - 2,3,2
314,32,2021-04-17 20:15:00,Molineux Stadium,Wolves,Sheffield Utd,1 - 0,1,0
315,32,2021-04-18 13:30:00,Emirates Stadium,Arsenal,Fulham,1 - 1,1,1
316,32,2021-04-18 16:00:00,Old Trafford,Man Utd,Burnley,3 - 1,3,1


In [42]:
df["Home Score"] = df["Home Score"].astype(int)
df["Away Score"] = df["Away Score"].astype(int)

In [67]:
def calculate_point(row_):
    if (row_["Home Score"] - row_["Away Score"]) < 0:
        return 0, 3
    elif (row_["Home Score"] - row_["Away Score"]) > 0:
        return 3, 0
    else:
        return 1, 1

In [71]:
result = df.apply(lambda x: calculate_point(x), axis=1).apply(pd.Series)
df = pd.concat([df, result], axis=1)
df = df.rename(columns={0: "Home Point", 1: "Away Point"})
df

,Round Number,Date,Location,Home Team,Away Team,Result,Home Score,Away Score,Home Point,Away Point
0,1,2020-12-09 12:30:00,Craven Cottage,Fulham,Arsenal,0 - 3,0,3,0,3
1,1,2020-12-09 15:00:00,Selhurst Park,Crystal Palace,Southampton,1 - 0,1,0,3,0
2,1,2020-12-09 17:30:00,Anfield,Liverpool,Leeds,4 - 3,4,3,3,0
3,1,2020-12-09 20:00:00,London Stadium,West Ham,Newcastle,0 - 2,0,2,0,3
4,1,2020-09-13 14:00:00,The Hawthorns,West Brom,Leicester,0 - 3,0,3,0,3
...,...,...,...,...,...,...,...,...,...,...
312,32,2021-04-17 12:30:00,St. James' Park,Newcastle,West Ham,3 - 2,3,2,3,0
314,32,2021-04-17 20:15:00,Molineux Stadium,Wolves,Sheffield Utd,1 - 0,1,0,3,0
315,32,2021-04-18 13:30:00,Emirates Stadium,Arsenal,Fulham,1 - 1,1,1,1,1
316,32,2021-04-18 16:00:00,Old Trafford,Man Utd,Burnley,3 - 1,3,1,3,0


In [58]:
calculate_point(df.iloc[0])

(0, 3)

In [55]:
df.iloc[0]

Round Number                      1
Date            2020-12-09 12:30:00
Location             Craven Cottage
Home Team                    Fulham
Away Team                   Arsenal
Result                        0 - 3
Home Score                        0
Away Score                        3
Name: 0, dtype: object